In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%env SEISMIQB_ANONYMIZE 1

import sys
import numpy as np
from copy import copy

PROJECT_PATH = '../' * 3
sys.path.insert(0, f'{PROJECT_PATH}seismiqb')
sys.path.insert(0, f'{PROJECT_PATH}src')

from seismiqb import SeismicDataset, HorizonMetrics, SeismicSampler
from seismiqb.batchflow import Pipeline

In [ ]:
from glob import glob
GEOMETRY_PATH = '/data/seismic_data/seismic_interpretation/001_YETYPUR/001_YETYPUR.blosc'
HORIZONS_PATH = glob('/data/seismic_data/seismic_interpretation/001_YETYPUR/INPUTS/HORIZONS/RAW/*')[1:4]

dataset = SeismicDataset({GEOMETRY_PATH: {'horizons': HORIZONS_PATH}})
print(dataset)

In [ ]:
field = dataset[0]
horizon = field.horizons[0]

In [ ]:
CROP_SHAPE = (512, 512, 1)

sampler = SeismicSampler(labels=dataset.labels, crop_shape=CROP_SHAPE, threshold=0., mode='horizon', shift_height=False)

sampler.show_sampled()

In [ ]:
BATCH_SIZE = 32

template = (
    Pipeline()
    .make_locations(generator=sampler, batch_size=BATCH_SIZE)
    .compute_label_attribute(src='horizons:*/amplitudes', dst='images', fill_value=0)
    .compute_label_attribute(src='horizons:*/masks', dst='masks', fill_value=0)
)
pipeline = template << dataset

In [ ]:
%%time
batch = pipeline.next_batch()
batch.show()

In [ ]:
batch.show_roll(n=5, components=[['images'], ['masks']])